In [1]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import cv2
import glob
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# Load the model
model = load_model('model/best_coin_recognition_model.h5')

In [8]:
# Prepare the image
imageOriginal = cv2.imread('data/test_images/4.jpg')
imageOriginal = cv2.resize(imageOriginal,(0,0), fx = 0.5, fy = 0.5)
image = imageOriginal.copy()
cv2.namedWindow("Initial", cv2.WINDOW_NORMAL)
cv2.imshow("Initial", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
image = cv2.GaussianBlur(image, (75, 75),1)
imgPaths = []

# Finding circles in the image
imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(imageGray, cv2.HOUGH_GRADIENT,1.3,200)
selectedCircles = []
print(circles)
i = 0
if circles is not None:
    # Get the (x, y, r) as integers
    circles = np.round(circles[0, :]).astype("int")
    for (x , y, r) in circles:
        i += 1
        if r < y and r < x:
            offset = r
            coinImage = image[y - offset: y + offset , x - offset : x + offset]
            coinOriginalImage = image[y - offset: y + offset , x - offset : x + offset]
            coinImage = cv2.resize(coinImage, (224,224))
            cv2.imwrite(f'temp_images/_{i}.jpg', coinImage)
            #cv2.imwrite(f'data/test_images/noise/_{i}.jpg', coinImage)
            imgPaths.append((f'temp_images/_{i}.jpg',(x, y, r)))

[[[ 460.84998   819.64996   101.27    ]
  [ 633.75     1090.0499    113.49    ]
  [ 165.75     1000.35      106.729996]
  [ 424.44998  1304.5499     96.590004]
  [ 766.35      775.44995   104.52    ]]]


In [9]:
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

classes = [25, 500, 0, 25,50,100]
# classes = [25, 25, 500, 25,100]
detectedCoinData = []
count = 0
for imgPath in imgPaths:
    count += 1
    image = Image.open(imgPath[0])
    #resize the image to a 224x224 with the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    # Load the image into the array
    data[0] = normalized_image_array
    cvImage = cv2.imread(imgPath[0])
    # run the inference
    prediction = model.predict(data)
    value = classes[np.argmax(prediction)]
    if(value != 0):
        detectedCoinData.append((cvImage, imgPath[1], value))
    # cv2.imshow(f'{count}: {value} ', cvImage)
    print(f'{count}:{classes[np.argmax(prediction)]}')
    
# cv2.waitKey(0)


1:100
2:25
3:25
4:50
5:100


In [10]:
# displaying the result
count25 = 0
count50 = 0
count100 = 0
count500 = 0
total = 0
for coin in detectedCoinData:
    r = coin[1][2]
    x = coin[1][0]
    y = coin[1][1]
    value = coin[2]
    if value == 25:
        count25 += 1
    if value == 50:
        count50 += 1
    if value == 100:
        count100 += 1
    if value == 500:
        count500 += 1
    cv2.rectangle(imageOriginal, (x - r, y - r), (x + r, y + r), (0, 255, 0),15)
    cv2.putText(imageOriginal, f'{value}', (x,y - r),fontFace=cv2.FONT_HERSHEY_SIMPLEX , fontScale=2 ,color=(0,0,255), thickness = 5)
    total += value
    
    
print(f'Total: {total} FCFA')
windowName = f'Total: {total} FCFA -- {count25} x 25, {count50} x 50, {count100} x 100, {count500} x 500 '
cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)
cv2.imshow(windowName, imageOriginal)
cv2.waitKey(0)
cv2.destroyAllWindows()

Total: 300 FCFA
